<a href="https://colab.research.google.com/github/caxios/PythonPlayground/blob/main/getFinanceData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def get_all_tickers(header):
  #It returns all comapies's tickers that is available from sec-api
  headers = {"User-Agent":f"{header}"}
  tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
  return tickers_cik.json()

In [ ]:
def get_financial_data(header, cik, us_gapp_element, sort_by):
  #It rerturns a timeseries pandas dataframe of seleceted us_gapp_element
  headers = {"User-Agent":header}
  response = requests.get(f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{us_gapp_element}.json", headers=headers)
  timeseries = pd.json_normalize(response.json()["units"]["USD"])
  timeseries["filed"] = pd.to_datetime(timeseries["filed"])
  timeseries = timeseries.sort_values(sort_by)
  timeseries = timeseries.iloc[::2, 7]
  return timeseries

In [ ]:
def show_plot(timeseries, x_sortBy, y_sortBy, plot_title, x_axis_name, y_axis_name):
  #It plots timeseries of given timeseries data.
  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(go.Scatter(
      x=timeseries[x_sortBy],
      y=timeseries[y_sortBy],
      name='Income (USD)',
  ))
  fig.update_layout(
      width=1000,
      height=700,
      paper_bgcolor='white',
      plot_bgcolor='#fafafa',
      hovermode='closest',
      title=plot_title,
      xaxis = dict(
          title=x_axis_name
      ),
      yaxis = dict(
          title=y_axis_name
      ),
      showlegend=False)
  fig.show()